In [1]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

import gensim.downloader
import numpy as np

from pathlib import Path

import glob

# 1. Чтение лемм

Дальнейшая разметка строится на предположении, что первое существительное в заголовке столбца отражает ту сущность, которая в нем содержится

In [2]:
df = pd.read_csv("../research/lemmatized/data.csv", sep=";")
df

,table_id,column_id,column_name_lemma
0,dataset/Dataset/data/7779522/table_0,0,чарт
1,dataset/Dataset/data/1888657/table_1,2,чарт
2,dataset/Dataset/data/6223308/table_0,0,чарт
3,dataset/Dataset/data/4293840/table_0,0,чарт
4,dataset/Dataset/data/8245389/table_1,0,чарт
...,...,...,...
3222710,dataset/Dataset/data/3660091/table_2,2,команда
3222711,dataset/Dataset/data/3660091/table_2,3,команда
3222712,dataset/Dataset/data/4212949/table_0,2,объем
3222713,dataset/Dataset/data/7525778/table_0,1,время


# **TODO:** use unique only in furhter labeling + join with all lemmas

In [3]:
unique_lemmas = pd.DataFrame()
unique_lemmas["column_name_lemma"] = pd.Series(df["column_name_lemma"].unique())
unique_lemmas

,column_name_lemma
0,чарт
1,высшаяпозиция
2,год
3,название
4,роль
...,...
13013,банрк
13014,субклад
13015,лагг
13016,ацетон


# 2. Чтение семантических типов

Некоторые типы были изменены, т.к. состояли из > 1 слова. Так как далее работаем с word2vec, эти типы были сокращены до одного слова.

In [4]:
labels = set()
with open("sem_types.txt", mode="r") as f:
    for line in f:
        if len(line.split(' ')) > 1:
            continue
        labels.add(line.lower().strip())

labels = list(labels)
labels.remove("мужчина")
labels.remove("женщина")
labels.remove("таблица")
labels.sort()

sem_types = dict((labels[i], i) for i in range(len(labels)))
sem_types

{'авиакомпания': 0,
 'автогонщик': 1,
 'автомобиль': 2,
 'агломерация': 3,
 'адрес': 4,
 'академик': 5,
 'актер': 6,
 'алгоритм': 7,
 'альбом': 8,
 'аниме': 9,
 'аннотация': 10,
 'арена': 11,
 'аристократ': 12,
 'археолог': 13,
 'архив': 14,
 'архитектор': 15,
 'астронавт': 16,
 'атлет': 17,
 'аффилиация': 18,
 'аэропорт': 19,
 'база': 20,
 'банк': 21,
 'баскетболист': 22,
 'батарея': 23,
 'башня': 24,
 'бейсболист': 25,
 'биатлонист': 26,
 'библиотека': 27,
 'бизнесмен': 28,
 'биомолекула': 29,
 'бог': 30,
 'бодибилдер': 31,
 'боксер': 32,
 'болезнь': 33,
 'больница': 34,
 'борец': 35,
 'браузер': 36,
 'вакцина': 37,
 'ввп': 38,
 'ведущий': 39,
 'велосипедист': 40,
 'вес': 41,
 'вещество': 42,
 'видеоигра': 43,
 'владелец': 44,
 'военный': 45,
 'возраст': 46,
 'вокзал': 47,
 'волейболист': 48,
 'вулкан': 49,
 'высота': 50,
 'газета': 51,
 'гендер': 52,
 'гимнаст': 53,
 'гитара': 54,
 'гитарист': 55,
 'глубина': 56,
 'год': 57,
 'гольфист': 58,
 'гонка': 59,
 'гора': 60,
 'город': 61,


# 3. Разметка

Каждому заголовку сопоставляется наиболее близкий семантический тип (метрика cosine similarity)

In [5]:
ru_model = gensim.downloader.load("word2vec-ruscorpora-300")

In [6]:
def get_most_similar(word: str) -> str|None:
    """Need to split afterwards because returns (label cos.sim.value)
    """

    max_similarity = 0
    max_similar_label = None
    
    try:
        ru_model.__getitem__([f"{word}_NOUN"])[0]
    except KeyError:
        return f"{max_similar_label} {max_similarity}"

    for label in sem_types.keys():
        try:
            similarity = ru_model.similarity(f"{label}_NOUN", f"{word}_NOUN")
        except KeyError:
            continue

        if similarity > max_similarity:
            max_similarity = similarity
            max_similar_label = label

    return f"{max_similar_label} {max_similarity}"

In [7]:
unique_lemmas["label"] = unique_lemmas["column_name_lemma"].progress_apply(lambda x: get_most_similar(x))
unique_lemmas

100%|████████████████████████████████████| 13018/13018 [00:17<00:00, 748.06it/s]


,column_name_lemma,label
0,чарт,сингл 0.5395997762680054
1,высшаяпозиция,None 0
2,год,год 0.9999999403953552
3,название,название 1.0
4,роль,актер 0.45205363631248474
...,...,...
13013,банрк,None 0
13014,субклад,None 0
13015,лагг,None 0
13016,ацетон,вещество 0.5003224015235901


In [8]:
df = df.merge(unique_lemmas, how="left")
df

,table_id,column_id,column_name_lemma,label
0,dataset/Dataset/data/7779522/table_0,0,чарт,сингл 0.5395997762680054
1,dataset/Dataset/data/1888657/table_1,2,чарт,сингл 0.5395997762680054
2,dataset/Dataset/data/6223308/table_0,0,чарт,сингл 0.5395997762680054
3,dataset/Dataset/data/4293840/table_0,0,чарт,сингл 0.5395997762680054
4,dataset/Dataset/data/8245389/table_1,0,чарт,сингл 0.5395997762680054
...,...,...,...,...
3222710,dataset/Dataset/data/3660091/table_2,2,команда,команда 1.0000001192092896
3222711,dataset/Dataset/data/3660091/table_2,3,команда,команда 1.0000001192092896
3222712,dataset/Dataset/data/4212949/table_0,2,объем,количество 0.5559502840042114
3222713,dataset/Dataset/data/7525778/table_0,1,время,период 0.4943215847015381


In [9]:
df[["label", "similarity"]] = df["label"].str.strip().str.split(" ", n=1, expand=True)
df

,table_id,column_id,column_name_lemma,label,similarity
0,dataset/Dataset/data/7779522/table_0,0,чарт,сингл,0.5395997762680054
1,dataset/Dataset/data/1888657/table_1,2,чарт,сингл,0.5395997762680054
2,dataset/Dataset/data/6223308/table_0,0,чарт,сингл,0.5395997762680054
3,dataset/Dataset/data/4293840/table_0,0,чарт,сингл,0.5395997762680054
4,dataset/Dataset/data/8245389/table_1,0,чарт,сингл,0.5395997762680054
...,...,...,...,...,...
3222710,dataset/Dataset/data/3660091/table_2,2,команда,команда,1.0000001192092896
3222711,dataset/Dataset/data/3660091/table_2,3,команда,команда,1.0000001192092896
3222712,dataset/Dataset/data/4212949/table_0,2,объем,количество,0.5559502840042114
3222713,dataset/Dataset/data/7525778/table_0,1,время,период,0.4943215847015381


In [10]:
empty_label_mask = (df["label"] == "None")
df[empty_label_mask]

,table_id,column_id,column_name_lemma,label,similarity
18702,dataset/Dataset/data/7779522/table_0,1,высшаяпозиция,None,0
18703,dataset/Dataset/data/4293840/table_0,1,высшаяпозиция,None,0
18704,dataset/Dataset/data/3215234/table_1,1,высшаяпозиция,None,0
18705,dataset/Dataset/data/962150/table_0,1,высшаяпозиция,None,0
18706,dataset/Dataset/data/6904957/table_1,1,высшаяпозиция,None,0
...,...,...,...,...,...
3222672,dataset/Dataset/data/1103808/table_1,6,фрегезии,None,0
3222673,dataset/Dataset/data/7008878/table_0,0,банрк,None,0
3222678,dataset/Dataset/data/4685610/table_0,3,субклад,None,0
3222698,dataset/Dataset/data/5273033/table_3,3,лагг,None,0


In [11]:
df = df[~empty_label_mask]
df

,table_id,column_id,column_name_lemma,label,similarity
0,dataset/Dataset/data/7779522/table_0,0,чарт,сингл,0.5395997762680054
1,dataset/Dataset/data/1888657/table_1,2,чарт,сингл,0.5395997762680054
2,dataset/Dataset/data/6223308/table_0,0,чарт,сингл,0.5395997762680054
3,dataset/Dataset/data/4293840/table_0,0,чарт,сингл,0.5395997762680054
4,dataset/Dataset/data/8245389/table_1,0,чарт,сингл,0.5395997762680054
...,...,...,...,...,...
3222710,dataset/Dataset/data/3660091/table_2,2,команда,команда,1.0000001192092896
3222711,dataset/Dataset/data/3660091/table_2,3,команда,команда,1.0000001192092896
3222712,dataset/Dataset/data/4212949/table_0,2,объем,количество,0.5559502840042114
3222713,dataset/Dataset/data/7525778/table_0,1,время,период,0.4943215847015381


## Выбор порогового значения

|порог|кол-во заголовков, схожесть >= порог|
|-----|------------------------------------|
|0.1|2.993.652|
|0.2|2.993.550|
|0.3|2.968.070|
|0.4|2.754.081|
|0.5|2.186.338|
|0.6|1.659.870|
|0.7|1.505.095|
|0.8|1.435.068|
|0.9|1.427.649|
|1.0|901.335|

P.s. Значения >= 1.0 за счет погрешности чисел с палвающей точкой

In [12]:
for i in range(1, 10+1):
    sim = (df["similarity"].apply(float) >= i / 10).sum()
    print(f"{i / 10} : {sim}")

0.1 : 3126334
0.2 : 3126232
0.3 : 3097159
0.4 : 2872031
0.5 : 2262380
0.6 : 1725406
0.7 : 1566161
0.8 : 1475735
0.9 : 1468204
1.0 : 940120


In [13]:
threshold = 0.78

df_threshold = df[df["similarity"].astype(float) > threshold]
df_threshold.sort_values("similarity")

,table_id,column_id,column_name_lemma,label,similarity
3133886,dataset/Dataset/data/7847449/table_0,0,мыслитель,философ,0.7839148044586182
3133884,dataset/Dataset/data/7846870/table_0,0,мыслитель,философ,0.7839148044586182
3133885,dataset/Dataset/data/8757853/table_0,0,мыслитель,философ,0.7839148044586182
2139300,dataset/Dataset/data/3414651/table_4,2,мотор,двигатель,0.7861505150794983
2139299,dataset/Dataset/data/1004039/table_4,4,мотор,двигатель,0.7861505150794983
...,...,...,...,...,...
813954,dataset/Dataset/data/2479117/table_4,1,команда,команда,1.0000001192092896
813955,dataset/Dataset/data/2706454/table_5,1,команда,команда,1.0000001192092896
813956,dataset/Dataset/data/4989442/table_1,1,команда,команда,1.0000001192092896
813948,dataset/Dataset/data/761249/table_0,0,команда,команда,1.0000001192092896


In [14]:
df_threshold["label"].value_counts()[:170]

label
год           252264
название      188881
место         117167
дата          107979
команда        82698
               ...  
колледж           93
префектура        93
борец             93
фирма             92
дорога            92
Name: count, Length: 170, dtype: int64

In [15]:
__mask = df_threshold["label"].value_counts()[:170].index

df_threshold = df_threshold[df_threshold["label"].isin(__mask)]
df_threshold

,table_id,column_id,column_name_lemma,label,similarity
24885,dataset/Dataset/data/4022432/table_0,0,год,год,0.9999999403953552
24886,dataset/Dataset/data/1777076/table_0,0,год,год,0.9999999403953552
24887,dataset/Dataset/data/143639/table_0,0,год,год,0.9999999403953552
24888,dataset/Dataset/data/1274811/table_2,0,год,год,0.9999999403953552
24889,dataset/Dataset/data/1250015/table_1,0,год,год,0.9999999403953552
...,...,...,...,...,...
3222708,dataset/Dataset/data/3660091/table_2,0,команда,команда,1.0000001192092896
3222709,dataset/Dataset/data/3660091/table_2,1,команда,команда,1.0000001192092896
3222710,dataset/Dataset/data/3660091/table_2,2,команда,команда,1.0000001192092896
3222711,dataset/Dataset/data/3660091/table_2,3,команда,команда,1.0000001192092896


In [16]:
df_threshold.sort_values("table_id", inplace=True)
df_threshold

,table_id,column_id,column_name_lemma,label,similarity
2366161,dataset/Dataset/data/1000006/table_0,0,театр,театр,1.0
2532049,dataset/Dataset/data/1000006/table_0,1,спектакль,пьеса,0.7898139357566833
1056778,dataset/Dataset/data/1000006/table_0,4,примечание,примечание,1.0000001192092896
1201015,dataset/Dataset/data/1000006/table_1,2,название,название,1.0
84764,dataset/Dataset/data/1000006/table_1,0,год,год,0.9999999403953552
...,...,...,...,...,...
750279,dataset/Dataset/data/999987/table_0,6,результат,результат,1.0
880312,dataset/Dataset/data/999987/table_0,2,турнир,турнир,1.0
164720,dataset/Dataset/data/999987/table_0,0,год,год,0.9999999403953552
539279,dataset/Dataset/data/999987/table_0,7,место,место,0.9999999403953552


In [17]:
df_threshold[df_threshold["similarity"].astype(float) >= 0.999]

,table_id,column_id,column_name_lemma,label,similarity
2366161,dataset/Dataset/data/1000006/table_0,0,театр,театр,1.0
1056778,dataset/Dataset/data/1000006/table_0,4,примечание,примечание,1.0000001192092896
1201015,dataset/Dataset/data/1000006/table_1,2,название,название,1.0
84764,dataset/Dataset/data/1000006/table_1,0,год,год,0.9999999403953552
2366198,dataset/Dataset/data/1000007/table_0,0,театр,театр,1.0
...,...,...,...,...,...
750279,dataset/Dataset/data/999987/table_0,6,результат,результат,1.0
880312,dataset/Dataset/data/999987/table_0,2,турнир,турнир,1.0
164720,dataset/Dataset/data/999987/table_0,0,год,год,0.9999999403953552
539279,dataset/Dataset/data/999987/table_0,7,место,место,0.9999999403953552


In [18]:
len(df_threshold["label"].unique())

170

In [19]:
df_threshold.to_csv(f"./labelled/threshold_{threshold}.csv", sep=";", index=False)